# Consolidator Playground

### Import Statements - some need to be installed separately from Anaconda base installation

In [56]:
import json
#import snowflake.connector
#pip install --upgrade snowflake-connector-python <- use this to install the snowflake connector package
#conda install -c conda-forge snowflake-connector-python
import pandas as pd
import csv
import fastparquet
from fastparquet import *
#Conda install fastparquet
import glob
import pyodbc
import xml.etree.ElementTree as ET
import nltk
#nltk.download('punkt')
import numpy as np
import random
import string
import datetime
import requests
import bs4 as bs
import urllib.request
import re
import pickle
print("Import Successful")

from collections import Counter


Import Successful


### Helpful Parsing Functions

In [2]:
def Replacy(a,b):
    while a in b:
        b = b.replace(a,'|')
    return b



def Splitty(x,y):
    SL = ['\\','.']
    for item in SL:
        x = Replacy(item,x)
        x = x.split('|')
    return x[y]

def GetTableName(f):
    fpath = str(f).split(".")[0]
    tablename = str(fpath.split("\\")[-2])
    return tablename

### Helpful baseline Load Functions for different File Types

In [3]:
def Load_CSV(f):
    with open(f, 'r') as r:
        S = [item.split(",")for item in r.readlines()]
    return S

def Load_SQL_DDL(f):
    with open(f, 'r') as r:
        S = r.readlines()
    return S

def Load_JSON(f):
    with open(f, 'r') as r:
        jmain = json.load(r)
    return jmain

def ParseXML(f,s):
    tree = ET.parse(f)
    root = tree.getroot()
    for child in root.iter('{http://www.mediawiki.org/xml/export-0.10/}text'):
        parse = child.text
    return parse.split(s)

In [4]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

def Get_Locations(f,start,end):
    #D = Load_CSV('Data\Site_Data.csv')
    D = Load_CSV(f)
    DL = [[line[-2],line[-1].replace('\n',''),line[0]] for line in D if line[5] == 'PJM' and line[-1] != "" and line[-2] != ""]
    DSList = []
    for line in DL:
        DS = Get_Location_Data(line[0],line[1],start,end)
        DSList.append([line[2],DS])
    return DSList

def Get_Location_Data(lat,long,start,end):
    data = '{ "lat": '+str(lat)+', "lon": '+str(long)+',"startDate": "'+str(start)+'","endDate": "'+str(end)+'" }'
    response = requests.post('http://weather.tceh.net/weather_historical_hourly/', headers=headers, data=data)
    r = response.json()
    return r
        
        
    
    

    
def Get_Station(lat,long,start,end):
    
    
    data = '{ "lat": '+str(lat)+', "lon": '+str(long)+',"startDate": "'+str(start)+'","endDate": "'+str(end)+'" }'
    response = requests.post('http://weather.tceh.net/weather_historical_hourly/', headers=headers, data=data)
    r = response.json()
    s = [r[0]['Latitude'],r[0]['Longitude']]
    return s

def Flatten_Data(DSList):
    klist = []
    klist.append('VistraSite')
    for d in DSList:
        for entry in d[1]:
            for k in entry:
                if k not in klist:
                    klist.append(k)
    flist = []
    #with open('WeatherDataForFred.csv', 'w') as export:
        #export.write(klist)
    flist.append(klist)
    for d in DSList:
        for entry in d[1]:
            alist = []
            alist.append(d[0])
            for k in klist[1:]:
                try:
                    alist.append(entry[k])
                except:
                    alist.append('NA')
            #export.write(alist)
            flist.append(alist)
    return flist


#Flatten_Data(Get_Data('38.95142','-76.77055','01/22/2020','01/24/2020'))

#for line in Flatten_Data(Get_Locations('Data\Site_Data.csv','01/22/2020','01/24/2020')):
    #print(line)

### Connecting to S3 and Snowflake

In [5]:
def Connections():
    print("Starting")
    ctx = snowflake.connector.connect(
        user='KWR1',
        password='dataflow1!',
        account='vistra.us-east-1.snowflakecomputing.com'
        )
    cs = ctx.cursor()
    try:
        cs.execute("USE DATABASE wholesale_dev;")
        one_row = cs.fetchone()
        print(one_row[0])
        
    finally:
        cs.close()
    ctx.close()
        
#Connections()

### Connecting to Vistra Databases

In [6]:
conn_a2 = pyodbc.connect('driver={SQL Server};Database=A2ISO;uid=svc_wdl_adapt2_pre-prod;pwd=Vistra567!;server=application.dev.dynegy.adapt2iso.com')
a2_cursor = conn_a2.cursor()

## Connection to NOOS not currently working
#pyodbc.drivers()
#connectString = pyodbc.connect('driver={ODBC Driver 17 for SQL Server};Database=NOOSTST3;uid=ARM8;pwd=N3ver#shgare;server=lmeae1nporalo01.tceh.net:1530')


print('Success')

Success


### Search for Tables and Return a list of all values in one specific attribute

In [7]:
def RefList(C,S,T,i):
    RL = []
    try:
        C.execute('SELECT * from '+S+'.'+T)
        data = list(C.fetchall())
        RL = [item[i] for item in data]
        return RL
    except:
        print("Error, could not execute query")
        
def Search(C,S,T):
    try:
        C.execute('SELECT * from '+S+'.'+T)
        print(T,"Column Descriptions:")
        desc = list(C.description)
        x = 0
        for d in desc:
            print(x, d)
            x += 1
        print("")
        print("Data:")
        cname = [item[0] for item in desc]
        print(cname)
        row = cursor.fetchmany(50)
        print(row)
    except:
        print("Error, could not execute query")

#RefList(a2_cursor, 'dbo', 'A2_AWARDS', 12)
#Search(a2_cursor,'dbo', 'PJM_MSRS_ENERGY_MKT_CONG_LOSS_CH_SUB')

### Print Table Attributes for Define Datapoints Document

In [8]:
def PTA(C,S,Tables):
    for T in Tables:
        C.execute('SELECT * from '+S+'.'+T)
        try:
            row = list(C.description)
            for item in row:
                Type = 'VARCHAR'
                Struc = 'X*'
                if str(item[1]) == "<class 'int'>":
                    Type = 'NUMBER'
                    Struc = '#*'
                elif str(item[1]) == "<class 'decimal.Decimal'>":
                    Type = 'NUMBER'
                    Struc = '#.#'
                elif str(item[1]) == "<class 'datetime.datetime'>":
                    Type = 'DATE'
                    Struc = 'mm/dd/yyyy' 
                print([T,item[0],Type,Struc])

    
                          
        except:
            pass
        
        
#PTA(a2_cursor,'dbo',['PJM_MSRS_ENERGY_MKT_CONG_LOSS_CH_SUB'])

### Import a Reference List and check that list against a database table

In [9]:
def LoadRef(f,i):
    R = Load_CSV(f)
    RList = [item[i] for item in R]
    return RList


def SearchForTables(C,S,Tables,RList):
    TL = []
    for T in Tables:
        C.execute('SELECT * from '+S+'.'+T)
        try:
            row = list(cursor.description)
            for item in row:
                if item[0].upper() in RList:
                    print([T,item[0]])

        
                          
        except:
            pass
    return TL

### Prep Database Table for DDL Export

In [10]:
def PrepTableDDL(C,S,T):
    C.execute('SELECT * from '+S+'.'+T)
    Atts = []
    try:
        row = list(C.description)
        for item in row:
            Type = 'VARCHAR'
            Struc = 'X*'
            if str(item[1]) == "<class 'int'>":
                Type = 'int'
                Struc = '#*'
            elif str(item[1]) == "<class 'decimal.Decimal'>":
                Type = 'int'
                Struc = '#.#'
            elif str(item[1]) == "<class 'datetime.datetime'>":
                Type = 'DATE'
                Struc = 'mm/dd/yyyy' 
            Atts.append([T,item[0],Type])
        
    except:
        pass
    
    return Atts



### Load Reference Tables and test the attributes in target files to see if the values are present in any reference attributes

In [11]:
def Get_Line(f,x):
    Header = str(f[0][x])
    l = [item[x] for item in f if item[x] != ""]
    l.pop(0)
    s = list(set(l))
    return [Header,s]

def Test_Line(l):
    reflist = []
    for f in glob.glob(r'Adapt2_DataPoints\Reference\*.csv'):
        Ref = Load_CSV(f)
        y = 0
        while y < len(Ref):
            try:
                C = Get_Line(Ref,y)
                Header = C[0]
                n = 0
                for a in l:
                    if a in C[1]:
                        n += 1
                if n > 0:
                    reflist.append([f,Header,n/len(l)])
                else:
                    pass
            except:
                pass
            y += 1
    return reflist
                
                
def Find_Refs(f):
    T = Load_CSV(f)

    x = 0
    while x < len(T[0]):
        try:
            L = Get_Line(T,x)
            Header = L[0]
            l = L[1]
            D = Test_Line(l)
            if D[2] != []:
                print(Header,D)
                print("")
        except:
            pass
        x += 1

def Ref_Match():
    for file in glob.glob('Adapt2_DataPoints\*\*.csv'):
        print(file)
        Find_Refs(file)
        print("")

### Parse Snowflake DDL to get a list of attributes

In [12]:
def Split_Tables(S):
    masterlist = []
    table = []
    for line in S:
        if 'CREATE OR REPLACE TABLE' in line:
            masterlist.append(table)
            table = []
            table.append(line)
        else:
            table.append(line)
    masterlist.append(table)
    masterlist.pop(0)
    for item in masterlist:
        for line in item:
            line = Replacy(' ',line)
            line = Replacy('(',line)
            line = Replacy(')',line)
            line = Replacy(',',line)
            line = Replacy('\t',line)
            line = Replacy('\n',line)
            line = line.split("|")
            while '' in line:
                line.remove('')
            if line == []:
                pass
            else:
                print(line)

### Parse a JSON File Recursively and return a list of available attribute layers

In [13]:
def Parse_Layer(l,n):
    kvl = [[key, value,n] for key, value in l.items()]
    return kvl

def Rec_PL(f):
    Layer_List = []
    try:
        Layer_List.append(Parse_Layer(Load_JSON(f),str(f)))
    except:
        Layer_List.append(Parse_Layer(Load_JSON(f)[0],str(f)))
    for x in Layer_List:
        for i in x:
            if type(i[1]) == list:
                a = 0
                while a < len(i[1]):
                    try:
                        Layer_List.append(Parse_Layer(i[1][a],i[0]))
                        #a += 1
                    except:
                        #print(a, 'failed')
                        #a += 1
                        continue
                    a += 1
    masterlist = []
    for d in Layer_List:
        typelist = []
        for item in d:
            typelist.append([item[2],item[0],str(type(item[1])).split("'")[1]])
        if typelist not in masterlist:
            masterlist.append(typelist)
            
    headers = []
    for item in masterlist:
        for h in item:
            if h[0] not in headers:
                headers.append(h[0])
    cleanlist = []
    for h in headers:
        uh = []
        for item in masterlist:
            for entry in item:
                if entry[0] == h:
                    if entry not in uh:
                        uh.append(entry)
        cleanlist.append(uh)
    u_entries_full = []
    for item in cleanlist:
        name = item[0][0]
        u_entries = []
        entries = list(set([entry[1] for entry in item]))
        for e in entries:
            types = [entry[2] for entry in item if entry[1] == e]
            u_entries.append([name,e,types])
        u_entries_full.append(u_entries)
    return u_entries_full

### Function for printing tables and DDL

In [14]:
def Create_Table_DDL(l,source):
    Directory = str('Create_Table_DDL\\'+source.upper()+"_"+"WDL_DDL.SQL")
    with open(Directory, 'w') as export:
        export.write('USE DATABASE wholesale_dev;')
        export.write('\n')
        export.write('USE SCHEMA master_confidential;')
        export.write('\n')
        for item in l:
            tablename = str(source.upper()+'_'+item[0][0].upper())

            export.write('CREATE OR REPLACE TABLE '+tablename+'_STG (')
            export.write("\n")
            for entry in item:
                try:
                    if entry[1].upper() == "SYSTEM_REFERENCE" or entry[1].upper() == "SORT_ORDER":
                        continue
                    else:
                        if 'int' in entry[2] or 'float' in entry[2] or 'NUMBER' in entry[2]:
                            export.write(entry[1].upper()+" NUMBER(38,2)")
                        elif 'bool' in entry[2] or 'BOOLEAN' in entry[2]:
                            export.write(entry[1].upper()+" BOOLEAN")
                        elif 'date' in entry[2] or 'TIMESTAMP' in entry[2]:
                            export.write(entry[1].upper()+" TIMESTAMP_LTZ")
                        else:
                            export.write(entry[1].upper()+" VARCHAR(50)")
                        export.write(",")
                        export.write("\n")
                except:
                    pass
            export.write("DWH_CREATION_TIME TIMESTAMP_NTZ,")
            export.write("\n")
            export.write("DWH_CREATION_USER  VARCHAR(50),")
            export.write("\n")
            export.write("DWH_UPDATED_TIME TIMESTAMP_NTZ,")
            export.write("\n")
            export.write("DWH_UPDATED_USER  VARCHAR(50),")
            export.write("\n")
            export.write("SOURCE_SYSTEM_KEY  NUMBER(38,2)")
            export.write("\n")
            export.write(");")
            export.write("\n")
            export.write("\n")
            
            


### Functions for Parsing Parquet files

In [75]:
def ParqParse(f):
    pf = ParquetFile(f)
    df = pf.to_pandas()
    fpath = str(f).split(".")[0]
    tablename = str(fpath.split("\\")[-2])
    table = []
    for col, d in zip(df.columns, df.dtypes):
        att = [tablename, col, str(d)]
        table.append(att)
    return table


def GetParqAtt(f):
    pf = ParquetFile(f)
    df = pf.to_pandas()
    fpath = str(f).split(".")[0]
    tablename = str(fpath.split("\\")[-2])
    table = []
    print(df.iloc[0])
    for col, d in zip(df.columns, df.dtypes):
        print([tablename, col, str(d)])
        

def Convert_Parq():  
    pass
    #for file in glob.glob(r'C:\Users\kwrs\Desktop\TalendDemo\pricing\backpack\test3\dbo\Stg_Mstar_List_Req_Data\*'):
        #pf = ParquetFile(file)
        #df = pf.to_pandas()
        #fpath = str(file).split(".")[0]
        #tablename = str(fpath.split("\\")[-1])
        #source = str(fpath.split("\\")[-2])
    
        #df.to_csv(r'C:\Users\kwrs\Desktop\TalendDemo\BACKPAC\NewStaging\\'+source+tablename+'.csv',index=False, sep = '|',quoting=csv.QUOTE_NONNUMERIC)
    
    #f = r'C:\Users\kwrs\Desktop\TalendDemo\pricing\backpack\test3\dbo\Transform_Config_Data\LOAD00000001.parquet'
    #pf = ParquetFile(f)
    #df = pf.to_pandas()
    #fpath = str(f).split(".")[0]
    #tablename = str(fpath.split("\\")[-2])






    #PriceIndex = list(set(df['PriceIndex'].tolist()))
    #for line in PriceIndex:
        #if '<year>' in line:
            #print(line)

    #is_ind = df["StgListReqDataID"]==1635911416
    #print(df[is_ind])
    #df.to_csv(r'C:\Users\kwrs\Desktop\TalendDemo\pricing\backpack\\'+tablename+'.csv',index=False, sep = '|',quoting=csv.QUOTE_NONNUMERIC)

def Load_Pipe_CSV(f):
    with open(f, 'r') as r:
        S = [item.split("|")for item in r.readlines()]
    return S

def Index_Counter():
    H = Load_Pipe_CSV(r'C:\Users\kwrs\Desktop\TalendDemo\BACKPAC\StageTest\Header\Stg_Mstar_List_Req_Hdr.csv')

    T = Load_Pipe_CSV(r'C:\Users\kwrs\Desktop\TalendDemo\BACKPAC\StageTest\Transform\Transform_Config_Data.csv')

    PI = list(set([item[6] for item in T]))

    LN = list(set([item[4] for item in T]))

    
    Index = list(set([(item[0],item[1]) for item in H]))
    cnt = Counter()
    for file in glob.glob(r'C:\Users\kwrs\Desktop\TalendDemo\BACKPAC\StageTest\Data\*'):
        
        D = Load_Pipe_CSV(file)
        D.pop(0)
        HC = [item[1] for item in D]
        for h in HC:
            cnt[h] += 1
    #print(cnt)
    with open('Data\Counter', 'wb') as WR:
        pickle.dump(cnt,WR)
    
#Index_Counter()

with open('Data\Counter', 'rb') as RR:
    cnt = pickle.load(RR)
    
#print(type(cnt))

#print(cnt.values())

H = Load_Pipe_CSV(r'C:\Users\kwrs\Desktop\TalendDemo\BACKPAC\StageTest\Header\Stg_Mstar_List_Req_Hdr.csv')
HH = H.pop(0)
#print(HH)
Index = [[item[1],item[0]] for item in H]
LN = list(set([item[0] for item in Index]))
IndexMaster = []
for l in LN:
    Hlist = [l,[item[1] for item in Index if item[0] == l]]
    IndexMaster.append(Hlist)
    
for i in IndexMaster:
    rows = []
    for c,v in zip(cnt.keys(),cnt.values()):
        if c in i[1]:
            rows.append(v)
    i.append(sum(rows))
for i in IndexMaster:
    print(i[0],":",i[2])

    

BackPac_Settled_NaturalGas : 9355
BackPac_Forward_ERCOT_Hourly_Other : 5358662
BackPac_WACOG_Coal : 0
BackPac_Settled_ERCOT2 : 0
BackPac_Forward_Emission_REC_Weather : 114792
BackPac_Forward_ERCOT_Hourly_LZ : 2622139
BackPac_Forward_ERCOT_Hourly_CoalPlant : 2116992
BackPac_Forward_Vol_Coal_Oil : 1926
BackPac_Settled_Oil : 383
BackPac_Settled_ExERCOT : 1429395
BackPac_Forward_ERCOT_Hourly_Hub : 3289914
BackPac_Settled_NaturalGas2 : 0
BackPac_Forward_PJM_Hourly : 2742070
BackPac_BPE_AS_Factors2 : 0
BackPac_Forward_ERCOT_Hourly_GasPlant : 3508767
BackPac_Forward_ERCOT : 123584
BackPac_Forward_InterestRate : 282880
BackPac_Settled_Oil2 : 0
BackPac_Forward_Coal : 33936
BackPac_BPE_AS_Factors : 90048
BackPac_Settled_ERCOT : 821583
BackPac_Forward_Oil_Nuclear : 7079
BackPac_CRR_Basis : 234
BackPac_BPE_Power_Factors : 387840
BackPac_Forward_NaturalGas_Options : 33637
BackPac_Forward_Vol_NaturalGas_Power : 152792
BackPac_Forward_ExERCOT : 34188
BackPac_Forward_NaturalGas : 44976
BackPac_Forward

### Prep Excel Mapping Spec for DDL Creation

In [16]:
def PrepExcel(f,X):

    x1 = pd.ExcelFile(f)
    Admin_sheets = ['Change Log', 'BlankDim', 'BlankFact','TEMPLATE']
    sheets = [name for name in x1.sheet_names if name not in Admin_sheets]
    FullDS = []
    for s in sheets:
        dataset = []
        att = []
        dfn = x1.parse(s)
        try:
            Table_Name = str(dfn.columns[1].replace(X,''))
            Table_Name = Table_Name.replace('_STG','')

            df = x1.parse(s,header=9)
            for row in df.iterrows():
                index, data = row
                dataset.append(data.tolist())
            for line in dataset:
                try:
                    if 'DWH_CREATION_TIME' in line or 'DWH_CREATION_USER' in line:
                        pass
                    else:
                    
                        att.append([Table_Name,line[0],line[3]])
                except:
                    pass
            FullDS.append(att)
        except:
            pass
    return FullDS

### Create Talend-Readable XMLs from Excel Mapping Specs

In [17]:
def Create_XMLs(f):
    
    TMDMAP = list(csv.reader(open(r'Talend_Metadata_Mapping.csv', 'rt'), delimiter=','))
    TMDMAP.pop(0)

    SFMD = list(set([item[2] for item in TMDMAP if item[2] != 'For Reference']))
    errorlog = []




    #dataset = []
    x1 = pd.ExcelFile(f)
    Admin_sheets = ['Change Log', 'BlankDim', 'BlankFact']
    sheets = [name for name in x1.sheet_names if name not in Admin_sheets]
    for s in sheets:
        dataset = []
        dfn = x1.parse(s)
        Table_Name = str(dfn.columns[1]+'.xml')
        Directory = str('XML_Mapping_Files\\'+Table_Name)

        df = x1.parse(s,header=9)
        for row in df.iterrows():
            index, data = row
            dataset.append(data.tolist())
        #dataset.pop(0)



        with open(Directory, 'w') as export:
            export.write('<?xml version="1.0" encoding="UTF-8"?><schema>')
    


            for line in dataset:
                if str(line[0]) == 'nan':
                    name = ""
                else:
                    name = line[0]
                if str(line[2]) == 'nan':
                    comment = ""
                else:
                    comment = line[2]
                datatype = ""
                originalDbColumnName = ""
                pattern = ""
                try:
                    if line[3] not in SFMD:
                        errorlog.append([Table_Name,line])
                        continue
                except:
                    continue
                else:
                    for entry in TMDMAP:
                        if entry[2] == line[3]:
                            datatype = entry[1]
                            originalDbColumnName = line[3]
                if datatype == 'id_Date':
                    pattern = '&quot;dd-MM-yyyy&quot;'
                if str(line[4]) == 'nan':
                    length = "-1"
                else:
                    length = str(line[4]).split('.')[0]
                if str(line[5]) == 'nan':
                    precision = "-1"
                else:
                    precision = str(line[5]).split('.')[0]
                key = 'false'
                if str(line[6]).upper() == 'Y':
                    key = 'true'
                nullable = 'true'
                if str(line[8]).upper() == 'N':
                    nullable = 'false'
                export.write('<column comment="'+comment+'" default="" key="'+key+'" label="'+name+'" length="'+length+'" nullable="'+nullable+'" originalDbColumnName="" originalLength="" pattern="'+pattern+'" precision="'+precision+'" talendType="'+datatype+'" type=""/>')
            export.write('</schema>')

    for line in errorlog:
        print(line)

### Working Space

In [18]:
def IsDate(x):
    try:
        t = datetime.datetime(x)
        return True
    except:
        return False
    
def IsFloat(x):
    try:
        t = float(x)
        return True
    except:
        return False
    
        
    
def ParseXML(f):
    tree = ET.parse(f)
    root = tree.getroot()
    #Attributes
    tl = list(set([item.tag for item in root.iter()]))
    for t in tl:
        #Data
        slist = list(set([item.text for item in root.iter(t)]))
        if t.split('}')[-1] == 'REPORT':
            NAME = slist[0]
    att_list = []
    for t in tl:
        slist = list(set([item.text for item in root.iter(t)]))
        tname = t.split('}')[-1]
        if slist != ['\n']:
            att = [NAME,tname]
            if IsDate(slist[0]) == True:
                att.append("TIMESTAMP")
            elif IsFloat(slist[0]) == True:
                att.append("NUMBER")
            else:
                att.append('STRING')
            att_list.append(att)
    return att_list


def Bulk_Create_DDL():
    L = []
    for file in glob.glob(r'Data\CAISO_MAG\*'):
        l = ParseXML(file)
        if l not in L:
            L.append(l)

    Create_Table_DDL(L,'CAISO_MA')
    
    
def Create_ATT_List():
    L = []
    for file in glob.glob(r'Data\CAISO_MAG\*'):
        l = ParseXML(file)
        for line in l:
            print(file,line)
        #if l not in L:
            #L.append(l)

    #Create_Table_DDL(L,'CAISO_MA')
    
#Create_ATT_List()
    

Create_Table_DDL(PrepExcel(r'Mapping_Spec\Adapt2 Mapping Spec.xlsm','ADAPT2_'),'ADAPT2')

### Working Space

In [19]:
#L = Load_CSV('Data\MISO_AO_DETERMINANTS_HOURLY_202002190913.csv')
#print(len(L))

#for line in L:
    #print([line[0],line[1].split(" at ")[0]])

### Load API

In [20]:
def Load_Wiki(w):
    raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/'+w)
    raw_html = raw_html.read()

    article_html = bs.BeautifulSoup(raw_html, 'lxml')
    article_paragraphs = article_html.find_all('p')
    article_text = ''

    for para in article_paragraphs:
        article_text += para.text

    article_text = article_text.lower()
    return article_text

### NLP-Based Fuzzy Matcher

In [21]:
def NG_Char(t, chars):
    ngrams = []
    for i in range(len(t)-chars+1):
        seq = t[i:i+chars]
        ngrams.append(seq)
    return ngrams

def NG_Char_Compare(s,t,chars):
    ng1 = NG_Char(s,chars)
    ng2 = NG_Char(t,chars)
    score = 0
    for k in ng1:
        if k in ng2:
            score += 1
    return score
    
def Predictor(s,tlist,chars):
    score = 0
    for t in tlist:
        val = NG_Char_Compare(s,t,chars)
        if val > score:
            score = val
            word = t
    return word
    
def NG_Char_Predictor(ngrams):
    curr_sequence = t[0:chars]
    output = curr_sequence
    for i in range(200):
        if curr_sequence not in ngrams.keys():
            break
        possible_chars = ngrams[curr_sequence]
        next_char = possible_chars[random.randrange(len(possible_chars))]
        output += next_char
        curr_sequence = output[len(output)-chars:len(output)]

    return output


def NG_Word(words):
    ngrams = {}


    words_tokens = nltk.word_tokenize(article_text)
    for i in range(len(words_tokens)-words):
        seq = ' '.join(words_tokens[i:i+words])
        print(seq)
        if  seq not in ngrams.keys():
            ngrams[seq] = []
        ngrams[seq].append(words_tokens[i+words])
    
    curr_sequence = ' '.join(words_tokens[0:words])
    output = curr_sequence
    for i in range(50):
        if curr_sequence not in ngrams.keys():
            break
        possible_words = ngrams[curr_sequence]
        next_word = possible_words[random.randrange(len(possible_words))]
        output += ' ' + next_word
        seq_words = nltk.word_tokenize(output)
        curr_sequence = ' '.join(seq_words[len(seq_words)-words:len(seq_words)])

    return output

### Footer to Make sure that a Run-All-Cells Completes

In [22]:
print("Successly Ran All Cells")

Successly Ran All Cells
